In [14]:
import numpy as np
import shapefile
import shapely
import shapely.geometry
from shapely.geometry import Polygon
from tqdm import tqdm
import pyproj
import scipy.sparse
from create_graph import create_graph
import geopandas as gpd

%load_ext autoreload    
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## SP Metropolis 10k Census Regions 

In [3]:
file = gpd.read_file("data/SP_Setores_2021/SP_Setores_2021.shp")
selected_row = 355030866000295
selected_row = file[file["CD_SETOR"].astype(int) == selected_row]
file["distance"] = file["geometry"].apply(lambda x: x.distance(selected_row.iloc[0]["geometry"]))
file = file.sort_values(by = "distance")
file = file.reset_index(drop = True)
file = file.head(10000)

In [4]:
file.NM_MUN.value_counts()

NM_MUN
São Paulo             9969
São Caetano do Sul      30
Guarulhos                1
Name: count, dtype: int64

In [5]:
file = file.drop(columns = ["distance"])
file.to_file("data/shapefiles/SpCenterCensus10k.shp")

In [6]:
file = gpd.read_file("data/shapefiles/SpCenterCensus10k.shp")
adj_matrix = create_graph(file)
adj_matrix = scipy.sparse.csr_matrix(adj_matrix)
scipy.sparse.save_npz("data/adj_matrix/SpCenterCensus10k.npz", adj_matrix)

100%|██████████| 10000/10000 [00:06<00:00, 1445.87it/s]


## SP Metropolis 20k Census Regions 

In [7]:
file = gpd.read_file("data/SP_Setores_2021/SP_Setores_2021.shp")
selected_row = 355030866000295
selected_row = file[file["CD_SETOR"].astype(int) == selected_row]
file["distance"] = file["geometry"].apply(lambda x: x.distance(selected_row.iloc[0]["geometry"]))
file = file.sort_values(by = "distance")
file = file.reset_index(drop = True)
file = file.head(20000)

In [8]:
file.NM_MUN.value_counts(), file.NM_MUN.unique()

(NM_MUN
 São Paulo                17355
 Guarulhos                  604
 Osasco                     559
 São Caetano do Sul         433
 Diadema                    348
 Santo André                294
 São Bernardo do Campo      217
 Taboão da Serra            160
 Mairiporã                   22
 Caieiras                     8
 Name: count, dtype: int64,
 array(['São Paulo', 'São Caetano do Sul', 'Guarulhos', 'Osasco',
        'São Bernardo do Campo', 'Santo André', 'Diadema', 'Mairiporã',
        'Taboão da Serra', 'Caieiras'], dtype=object))

In [9]:
file = file.drop(columns = ["distance"])
file.to_file("data/shapefiles/SpCenterCensus20k.shp")

In [10]:
file = gpd.read_file("data/shapefiles/SpCenterCensus20k.shp")
adj_matrix = create_graph(file)
adj_matrix = scipy.sparse.csr_matrix(adj_matrix)
scipy.sparse.save_npz("data/adj_matrix/SpCenterCensus20k.npz", adj_matrix)

100%|██████████| 20000/20000 [00:14<00:00, 1388.08it/s]


In [11]:
region = file.geometry.unary_union
region = region.simplify(0.01)
gdf = gpd.GeoDataFrame(geometry=[region])
gdf.to_file("data/shapefiles/SpCenterCensus20k_countour.geojson", driver="GeoJSON")

## Districts

In [19]:
file = gpd.read_file("data/LAYER_DISTRITO/DEINFO_DISTRITO.shp")
file = file.to_crs("EPSG:4326")
file.to_file("data/shapefiles/SpDistricts.shp")

In [21]:
shapefile = gpd.read_file("data/shapefiles/SpDistricts.shp")
adj_matrix = create_graph(shapefile)
np.save("data/adj_matrix/SpDistricts.npy", adj_matrix)

100%|██████████| 96/96 [00:00<00:00, 238.77it/s]
